In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import pyspark.sql.functions as F

In [2]:
credentials_location = '/home/yqian/de-zoomcampfire-453e19e2c7e3.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/yqian/de_zoomcamp/spark_lib/gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/02/28 03:28:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
df_yellow = spark.read.parquet("gs://zoomcamp-misc/yellow_tripdata/2024/yellow_tripdata_2024-10.parquet")
print(df_yellow.count())

3833771


In [6]:
# Question 2
df_yellow \
    .repartition(4) \
    .write \
    .parquet('data/', mode='overwrite')

In [7]:
# Question 3
df_yellow = df_yellow.withColumn('pickup_date', df_yellow['tpep_pickup_datetime'].cast('date'))
print(df_yellow.filter(df_yellow['pickup_date'] == '2024-10-15').count())

128893


In [8]:
# Question 4
df_yellow = df_yellow.withColumn(
    'trip_time', 
    (F.unix_timestamp('tpep_dropoff_datetime') - F.unix_timestamp('tpep_pickup_datetime')) / 60 / 60)
print(df_yellow.orderBy(F.desc('trip_time')).first()['trip_time'])

162.6177777777778


In [9]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
df_zone = spark.read.option("header", "true").csv("taxi_zone_lookup.csv")
df_zone.printSchema()

--2025-02-28 03:29:26--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.154.99.225, 18.154.99.177, 18.154.99.220, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.154.99.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-02-28 03:29:26 (238 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [10]:
# Question 6
df_join = df_yellow.join(df_zone, df_yellow['PULocationID'] == df_zone['LocationID'], how='left')
df_join = df_join.groupBy("Zone").count().orderBy('count')
print(df_join.first())

Row(Zone="Governor's Island/Ellis Island/Liberty Island", count=1)
